In [64]:
# Imports
import numpy as np

In [113]:
# Import the tools
from sys import path
# path.append('/Users/reid/dev/PythonCode/tm/tools')
path.append('D:/PythonCode/tm/tools')
from tools import gen_inputs_outputs, logsig, lin, deriv_logsig, deriv_lin, e2

In [66]:
# Define the size of the network
# Order by layers (input, layer1, layer2, ...)
# !!! No output layer yet, wait to gen inputs
layer_sizes = [3, 7]

In [67]:
# Gen inputs, outputs and size of the last layer
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

In [68]:
# Append the output layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [69]:
# Network shape
layer_sizes

array([3, 7, 3])

In [70]:
#####
# layer_sizes = np.array([1,2,1])

In [71]:
# Inputs
inputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

In [72]:
# Outputs
outputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [73]:
# Intialize the weights and biases randomly
# Also create the list to store the outputs of each layer
# Store the outptut for each layer
# Finally create the list of the sensitivities
weights_list = []
biases_list = []
n_list = []
a_list = []
s_list = []

In [117]:
# Define a function to reset all the parameters
# This is done to test multiple different network types
def reset_params():
    weights_list.clear()
    biases_list.clear()
    n_list.clear()
    a_list.clear()
    s_list.clear()
    for i in range(1, len(layer_sizes)):
        weights_list.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))
        biases_list.append(np.random.rand(layer_sizes[i], 1))
        n_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        a_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        s_list.append(np.empty((layer_sizes[i], 1), dtype=float))

In [75]:
#####
'''weights_list = [
    np.array([[-0.27],[-0.41]]),
    np.array([[0.09, -0.17]])
]
biases_list = [
    np.array([[-0.48], [-0.13]]),
    np.array([[0.48]])
]
n_list = [
    np.array([[0], [0]]),
    np.array([[0]])
]
a_list = [
    np.array([[0], [0]]),
    np.array([[0]])
]'''

'weights_list = [\n    np.array([[-0.27],[-0.41]]),\n    np.array([[0.09, -0.17]])\n]\nbiases_list = [\n    np.array([[-0.48], [-0.13]]),\n    np.array([[0.48]])\n]\nn_list = [\n    np.array([[0], [0]]),\n    np.array([[0]])\n]\na_list = [\n    np.array([[0], [0]]),\n    np.array([[0]])\n]'

In [76]:
# Define the transfer functions
transfer_functions = [logsig, lin]

In [77]:
# Define the vectors with the derivatives of the transfer functions
# These need to be converted to dialation matrices
# but this is done after the partial derivatives are calculated
# so that the numerical values can be multiplied by the identity matrix
# makes my life easier
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [78]:
# Define a function to calculate the output of the network
# while also saving the data
# Assuming that the input is already a Rx1 matrix
def run_network(X):
    # Get the input for the first layer so that
    # no need to use X and can calculate
    # recursively
    n_list[0] = np.matmul(weights_list[0], X) + biases_list[0]
    a_list[0] = transfer_functions[0](n_list[0])

    # Calculate the rest of the outptut
    for i in range(1, len(weights_list)):
        n_list[i] = np.matmul(weights_list[i], a_list[i-1]) + biases_list[i]
        a_list[i] = transfer_functions[i](n_list[i])

In [79]:
# Calculate the senstivites for the last layer
# Write a function to fill the diagonal of the matrix
def fill_F_dot(layer_num):
    a = np.identity(len(n_list[layer_num]), float)
    np.fill_diagonal(a, deriv_transfer_functions[layer_num](n_list[layer_num]).flatten())
    return a

def train_network(learning_rate=.1):
    # Iterate through the inputs and outputs
    for i in range(len(inputs)):
        # Run the network
        run_network(inputs[i].reshape(layer_sizes[0], 1))

        # Get the dialation matrix for the last layer
        F_dot = fill_F_dot(-1)

        # Calculate the sensitivites for the last layer
        s_list[-1] = -2 * np.matmul(F_dot, outputs[i].reshape(layer_sizes[-1], 1)-a_list[-1])

        # Update the weights and biases for the last layer
        weights_list[-1] += -learning_rate*np.matmul(s_list[-1], a_list[-2].T)
        biases_list[-1] += -learning_rate*s_list[-1]

        # Iterate through the remaining layers backwards
        for i in range(len(n_list)-2, 0, -1):
            s_list[i] = np.matmul(np.matmul(fill_F_dot(i), weights_list[i+1].T), s_list[i+1])
            weights_list[i] += -learning_rate*np.matmul(s_list[i], a_list[i-1].T)
            biases_list[i] += -learning_rate*s_list[i]

In [115]:
# Function to evaluate the performance
# of the network
def performance():
    results = np.empty((len(inputs), layer_sizes[-1]), dtype=float)
    for i in range(len(inputs)):
        run_network(inputs[i].reshape(layer_sizes[0], 1))
        results[i] = outputs[i] - a_list[-1].flatten()
    print(e2(results), results)

In [171]:
reset_params()
train_network(learning_rate=.1)
performance()

20.5887428552656 [[-0.90411312  0.12435177  0.18652744]
 [-1.07417591  0.20440762  1.27514445]
 [-0.96565207  1.12379037  0.17061849]
 [-1.11602031  1.1932689   1.2429933 ]
 [-1.09867778  0.1859957   1.25377207]
 [-1.21942603  1.2566614   0.32205169]
 [-1.14338234  1.19126762  1.24154456]
 [-0.24355668  0.25113091  0.29445738]]


In [13]:
# Test the model with keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [14]:
model = Sequential()
model.add(Dense(layer_sizes[1], input_shape=(layer_sizes[0],), activation='sigmoid'))
model.add(Dense(layer_sizes[2], activation='linear'))

In [15]:
sdg = SGD(.9)
model.compile(loss="categorical_crossentropy", optimizer=sdg,
	metrics=["accuracy"])
model.fit(inputs, outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.5000 - loss: 7.0425


In [16]:
model.predict(inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[ 0.01186371, -8.282264  , -1.4656483 ],
       [ 0.0887537 , -9.469966  , -1.5549538 ],
       [ 0.15937938, -8.323184  , -1.3819622 ],
       [ 0.22767666, -9.4916    , -1.4718292 ],
       [-0.01628557, -8.611647  , -1.6199257 ],
       [ 0.04203045, -9.72632   , -1.6907108 ],
       [ 0.14307892, -8.662352  , -1.5352875 ],
       [ 0.20412186, -9.765932  , -1.603532  ]], dtype=float32)